In [ ]:
# # This cell runs bowtie locally, uncomment last line to run the command and test your bowtie
# 
# import os
# # wget index and SRR043378 directly into project folder
# threads = 4  # Can be changed
# # folder name then file name
# bowtieIndex = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
#               "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"
# # fastq read file
# reads = "SRR043378_1.fastq/data"
# cmd = "bowtie2 -p "+str(threads)+" -x " + bowtieIndex + " -U "+reads+" -S out.sam"
# 
# 
# os.system(cmd)

In [5]:
# # This cell tests pyspark
# 
# import pyspark
# import random
# sc = pyspark.SparkContext(appName="Pi")
# num_samples = 100000000
# 
# 
# def inside(p):     
#     x, y = random.random(), random.random()
#     return x*x + y*y < 1
# 
# 
# count = sc.parallelize(range(0, num_samples)).filter(inside).count()  
# pi = 4 * count / num_samples
# print(pi)
# sc.stop()

3.1413872


In [1]:
# import pyspark
# import os
# 
# # new sparkContext
# sc = pyspark.SparkContext(appName="DNA")

In [2]:
#######################################################################################################
# TODO: Put files in hdfs, split file into parts for each worker to process (should be able to just run
# bowtie2 command on each mapper node). Then run mapper, should output however many files there are 
# nodes, then combine the output files. 
#######################################################################################################

 
# # Basis for map function, output files probably need different names for different nodes and file 
# # locations need to be changed to the hdfs location
# threads = 1  # Can be changed
# # folder name then file name
# bowtieIndex = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
#               "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"
# # fastq read file
# reads = "SRR043378_1.fastq/data"
# cmd = "bowtie2 -p " + str(threads) + " -x " + bowtieIndex + " -U " + reads + " -S out.sam"


In [1]:
#import findspark
#findspark.init('/home/taylor/spark')

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext


conf = SparkConf().setMaster("local").setAppName("DNA")

sc = SparkContext(conf=conf)


In [2]:
raw_input = (sc.textFile("SRR043378_1.fastq/data")).zipWithIndex()


In [3]:
raw_input.take(10)

[('@SRR043378.1 618UWAAXX100205:6:1:995:4021/1', 0),
 ('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN',
  1),
 ('+', 2),
 ('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!',
  3),
 ('@SRR043378.2 618UWAAXX100205:6:1:996:5311/1', 4),
 ('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN',
  5),
 ('+', 6),
 ('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!',
  7),
 ('@SRR043378.3 618UWAAXX100205:6:1:996:10707/1', 8),
 ('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN',
  9)]

In [9]:
import math
indexed_raw_input = raw_input.map(lambda x: (math.floor(x[1]/4), x[0]))

In [10]:
indexed_raw_input.take(10)

[(0, '@SRR043378.1 618UWAAXX100205:6:1:995:4021/1'),
 (0,
  'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'),
 (0, '+'),
 (0,
  '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'),
 (1, '@SRR043378.2 618UWAAXX100205:6:1:996:5311/1'),
 (1,
  'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'),
 (1, '+'),
 (1,
  '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'),
 (2, '@SRR043378.3 618UWAAXX100205:6:1:996:10707/1'),
 (2,
  'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN')]

In [13]:
reads = indexed_raw_input.groupByKey()#.mapValues(lambda x1, x2, x3, x4: x1+x2+x3+x4)

In [16]:
reads.take(10)


In [22]:
#reads.mapValues(list).take(10)
#reads.reduce(lambda x: x[1]+x[2]+x[3]+x[4])


In [19]:
reads.take(10)


In [ ]:
sc.stop()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:36013)
Traceback (most recent call last):
  File "/home/taylor/PycharmProjects/DNA_Project/venv/lib/python3.6/site-packages/py4j/java_gateway.py", line 852, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/taylor/PycharmProjects/DNA_Project/venv/lib/python3.6/site-packages/py4j/java_gateway.py", line 990, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


/home/taylor/PycharmProjects/DNA_Project/venv/lib/python3.6/site-packages/pyspark/context.py:410: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.
  RuntimeWarning
